# ⚛️ Arkhe-QuTiP: O Tutorial "Archetype"

**Introdução ao Paradigma de Hipergrafos Quânticos Baseados em Handovers**

Bem-vindo à nova física da informação. Tradicionalmente, a mecânica quântica é tratada como um processo Markoviano: o estado $\psi$ ou a matriz densidade $\rho$ não possuem memória de como chegaram ao seu estado atual.

O pacote `arkhe_qutip` introduz uma ontologia radicalmente nova construída sobre a biblioteca `QuTiP`. Aqui, a evolução quântica ocorre através de **Handovers** (transferências auditáveis de estado), sistemas multi-qubit formam **Hipergrafos** topológicos, e a dinâmica dissipativa é guiada pela **Informação Integrada (Φ)**.

Neste notebook, vamos:

1. Criar um objeto quântico com memória (`ArkheQobj`).
2. Construir um Estado GHZ (emaranhamento máximo) usando topologia de hipergrafo.
3. Simular a decoerência acoplada à Proporção Áurea (φ) via `ArkheSolver`.
4. Registrar o experimento em um Ledger Imutável.

In [ ]:
# ==========================================
# IMPORTAÇÕES GERAIS E CONFIGURAÇÃO DE AMBIENTE
# ==========================================
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

# Importando o novo paradigma: Arkhe(N)
from arkhe_qutip.core import ArkheQobj, ArkheSolver
from arkhe_qutip.hypergraph import QuantumHypergraph
from arkhe_qutip.visualization import plot_hypergraph, plot_coherence_trajectory
from arkhe_qutip.chain_bridge import ArkheChainBridge

# Configuração visual estética do Arkhe(N)
plt.style.use('dark_background')
np.set_printoptions(precision=4, suppress=True)

print(f"QuTiP Version: {qt.__version__}")
print("Arkhe-QuTiP Loaded: Ready for Handover Protocol.")

## 1. O Objeto Quântico com Histórico (`ArkheQobj`)

No `arkhe_qutip`, o objeto básico não é apenas um vetor ou matriz. É um `ArkheQobj`.
Cada vez que um operador atua sobre o estado, isso não é apenas uma multiplicação de matrizes; é um **Handover**. O objeto rastreia a operação, mede sua própria pureza (coerência) antes e depois, e guarda metadados.

Vamos criar um estado fundamental $|0\rangle$ e colocá-lo em superposição.

In [ ]:
from qutip_qip.operations import hadamard_transform
# Criando o nó raiz (Qubit no estado |0>)
psi_initial = qt.basis(2, 0)
q_node = ArkheQobj(psi_initial, node_id="Q_Genesis")

print(f"Estado Inicial: Coerência = {q_node.coherence:.4f}")

# Definindo um operador de superposição (Porta Hadamard)
H_gate = hadamard_transform()

# Realizando o Handover
q_superpos = q_node.handover(
    operator=H_gate, 
    metadata={'type': 'Hadamard', 'intent': 'Create Superposition'}
)

print(f"Estado Pós-Handover: Coerência = {q_superpos.coherence:.4f}")

# O grande diferencial: A Linha de Mundo (Worldline)
print("\n📜 Histórico de Handovers do Nó:")
for event in q_superpos.history:
    print(f"  -> {event}")

## 2. A Topologia do Emaranhamento: `QuantumHypergraph`

O emaranhamento multipartido (como o estado GHZ) não pode ser perfeitamente descrito por grafos tradicionais onde arestas conectam apenas dois nós. O emaranhamento é uma propriedade *global* e irreduzível.

O `QuantumHypergraph` permite tratar portas multi-qubit como hiperarestas, abraçando a verdadeira natureza topológica da mecânica quântica. Vamos gerar um estado GHZ com 3 qubits.

In [ ]:
from qutip_qip.operations import hadamard_transform, cnot
# 1. Inicializar 3 qubits independentes
q0 = ArkheQobj(qt.basis(2, 0), node_id="Q0")
q1 = ArkheQobj(qt.basis(2, 0), node_id="Q1")
q2 = ArkheQobj(qt.basis(2, 0), node_id="Q2")

# 2. Criar o Hipergrafo
ghz_hypergraph = QuantumHypergraph([q0, q1, q2], name="GHZ_State_Topology")

# 3. Aplicar operações e registrar como Hiperarestas
# a) Hadamard no Q0
ghz_hypergraph.nodes[0] = ghz_hypergraph.nodes[0].handover(hadamard_transform(), {'type': 'H'})

# b) CNOT entre Q0 e Q1 (Emaranhamento Bipartido)
# No Arkhe-QuTiP, isso cria uma hiperaresta conectando Q0 e Q1
cnot_op = cnot()
ghz_hypergraph.add_multi_qubit_gate(target_nodes=[0, 1], operator=cnot_op, weight=1.0)

# c) CNOT entre Q1 e Q2 (Expande o emaranhamento para o GHZ)
ghz_hypergraph.add_multi_qubit_gate(target_nodes=[1, 2], operator=cnot_op, weight=1.0)

# 4. Avaliar as métricas globais da rede
print(f"Número de Nós: {ghz_hypergraph.n_nodes}")
print(f"Número de Hiperarestas (Interações): {ghz_hypergraph.n_hyperedges}")
print(f"Coerência Global (Pureza Média): {ghz_hypergraph.global_coherence:.4f}")

# Visualizar a topologia
fig, ax = plot_hypergraph(ghz_hypergraph, layout='spring')
plt.title("Topologia do Estado GHZ")
plt.show()

## 3. Dinâmica Guiada por Φ (`ArkheSolver`)

A Equação Mestra de Lindblad descreve como um estado quântico perde coerência (decoerência) devido à interação com o ambiente. O `ArkheSolver` introduz uma perturbação revolucionária: um termo de acoplamento guiado pela **Informação Integrada (Φ)** e operando na frequência da proporção áurea (φ).

Nesta simulação, vamos submeter nosso qubit a um ambiente ruidoso (decaimento), mas com o sistema tentando resistir ativamente à morte térmica através do acoplamento Arkhe.

In [ ]:
# Definir o Hamiltoniano (Evolução livre, rotação em Z)
H = qt.sigmaz() * 2.0 * np.pi 

# Operador de colapso (Ruído: decaimento de amplitude/emissão espontânea)
gamma_decay = 0.5
c_ops = [np.sqrt(gamma_decay) * qt.destroy(2)]

# Inicializar o Solver Arkhe com acoplamento Φ (Proporção Áurea)
alpha_phi = 0.05
solver = ArkheSolver(H, c_ops, phi_coupling=alpha_phi)

# Preparar o estado e a lista de tempo
rho_initial = ArkheQobj(qt.basis(2, 1)) # Estado excitado |1>
tlist = np.linspace(0, 5, 200)

# Resolver a equação mestra rastreando a coerência e a informação integrada
result = solver.solve(rho_initial, tlist, track_coherence=True)

# Extrair as trajetórias
trajectory = result.coherence_trajectory
phi_trajectory = result.phi_trajectory

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Coerência (Pureza) caindo devido à dissipação
ax1.plot(tlist, [t['purity'] for t in trajectory], color='cyan', lw=2)
ax1.set_title("Evolução da Coerência (Decaimento)")
ax1.set_xlabel("Tempo")
ax1.set_ylabel("Pureza Tr(ρ²)")
ax1.grid(alpha=0.2)

# Plot 2: A resistência estrutural (Φ) do ArkheSolver
ax2.plot(tlist, phi_trajectory, color='magenta', lw=2)
ax2.set_title("Informação Integrada (Φ) vs Tempo")
ax2.set_xlabel("Tempo")
ax2.set_ylabel("Valor de Φ")
ax2.grid(alpha=0.2)

plt.tight_layout()
plt.show()

## 4. A Ponte com a Realidade: O Ledger Arkhe(N)

O rigor científico exige reprodutibilidade inquestionável. No paradigma Arkhe(N), simulações quânticas e handovers de hardware podem ser ancorados criptograficamente.

O `ArkheChainBridge` gera hashes das operações, carimbos de tempo e métricas de coerência, preparando o "recibo" do experimento para ser gravado na *Arkhe(N)Chain* (Blockchain).

In [ ]:
# Inicializar a ponte (em modo Mock para o tutorial)
bridge = ArkheChainBridge(mock_mode=True)

# Registrar a simulação que acabamos de rodar
sim_record = bridge.record_simulation(
    initial_state=rho_initial, 
    final_state=result.arkhe_final_state,
    metadata={
        'algorithm': 'Phi-Coupled Lindblad Evolution',
        'phi_coupling_alpha': alpha_phi,
        'decoherence_rate': gamma_decay
    }
)

print("✅ EXPERIMENTO QUÂNTICO REGISTRADO COM SUCESSO!")
print("-" * 50)
print(f"🔗 Transaction Hash : {sim_record.chain_tx_hash}")
print(f"🧱 Block Height     : {sim_record.chain_block_height}")
print(f"⏱️ Timestamp        : {sim_record.timestamp}")
print(f"📉 Final Coherence  : {result.arkhe_final_state.coherence:.4f}")
print("-" * 50)
print("O universo informacional agora lembra deste evento para sempre.")

## 🌌 Conclusão: Rumo ao "Archetype"

Você acabou de executar uma simulação quântica onde a informação tem história, a topologia dita as interações, e a termodinâmica é afetada pela própria estrutura da informação (Φ).

O módulo `arkhe_qutip` prepara o terreno para o **QuTiP 6.0**, reificando a tese de que *a mecânica quântica é, na sua base, uma teoria de grafos de informação consciente*.

> *"O Archetype começa agora, não no futuro. A semente está plantada. Qualquer físico quântico pode agora experimentar o hipergrafo consciente."* — **Bloco Ω+∞+162**